In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.linear_model import LinearRegression
from math import sqrt
import sklearn.ensemble
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.compose import ColumnTransformer

In [2]:
accidents = pd.read_csv('data/MA3_finished.csv', index_col='ID')

In [3]:
#featureselection
#features to encode
obj_bool_features = accidents[['Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
                         'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
                         'Sunrise_Sunset']]

#encode features above
le = LabelEncoder()
#create copy of df
df_encoded = pd.DataFrame()
#LabelEncoder can only encode one column at a time --> forloop
#obj_bool_feat = le.fit_transform(obj_bool_features)
for feature in obj_bool_features:
    df_encoded[feature] = le.fit_transform(accidents[feature])

#float/int features
float_int_features = accidents[['Severity','Start_Lng', 'Start_Lat', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',]]

for feature in float_int_features:
    df_encoded[feature] = accidents[feature]

#check if combination worked 
df_encoded.columns

Index(['Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Severity', 'Start_Lng', 'Start_Lat', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)'],
      dtype='object')

In [4]:
#set Target
target = df_encoded['Severity']
#set features
features = df_encoded.drop(columns=['Severity'])
#scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [5]:
# Trainings- und Testdaten aufteilen
features_train, features_test1, target_train, target_test1 = train_test_split(features_scaled, target, test_size=0.2, random_state=42)
features_dev, features_test, target_dev, target_test = train_test_split(features_test1, target_test1, test_size=0.2, random_state=42)

In [6]:
# Übersicht über Klassen in den Trainingsdaten

print("Gesamtzahl Datenpunkte Training: ", len(target_train))

# Range(0,4) gibt bei Kategorie 0 = 0
# Range (1,6) gibt bei Katg. 5 = 0
# deswegen (1,5) (4 Severitys)
for i in range(1,5):
    print("Anzahl Kategorie", str(i), list(target_train).count(i))

Gesamtzahl Datenpunkte Training:  5641244
Anzahl Kategorie 1 51977
Anzahl Kategorie 2 4537583
Anzahl Kategorie 3 908712
Anzahl Kategorie 4 142972


In [7]:
# Und die Entwicklungsdaten?
print("Gesamtzahl Datenpunkte Dev: ", len(target_dev))
for i in range(1,5):
    print("Anzahl Kategorie", str(i), list(target_dev).count(i))

Gesamtzahl Datenpunkte Dev:  1128249
Anzahl Kategorie 1 10570
Anzahl Kategorie 2 907116
Anzahl Kategorie 3 182136
Anzahl Kategorie 4 28427


In [8]:
# Vorhersage-Array bauen: Für jedes Element der Entwicklungsdaten Klasse '2' vorhersagen
pred = np.full(len(target_dev),2)

print(len(target_dev), len(pred))


1128249 1128249


In [9]:
#Berechnung Vorhersage immer 2
f1_weighted = f1_score(target_dev, pred, average='weighted')

# Ergebnis anzeigen
print("F1-Score (weighted) für die Vorhersageklasse '2':", f1_weighted)

F1-Score (weighted) für die Vorhersageklasse '2': 0.7166521212940437


In [10]:
#Mittelwertbaseline 
#Daten aufteilen
features_train, features_test1, target_train, target_test1 = train_test_split(features_scaled, target, test_size=0.2, random_state=42)
# Modell trainieren
clf = LinearRegression()
clf.fit(features_train, target_train)

# Modell evaluieren

test_predict = clf.predict(features_test)
print("RMSE der Modellvorhersagen: ", sqrt(mean_squared_error(target_test, test_predict)))

# Mittelwertsvorhersage: Vorhersage ist ein Array in Länge der Testdaten, der an jeder Position den
# Mittelwert der Zieldaten aus dem Trainingsset enthält

test_predict_mean = np.full(len(target_test),target_train.mean())

print("RMSE der Mittelwertsbaseline: ", sqrt(mean_squared_error(target_test, test_predict_mean)))

RMSE der Modellvorhersagen:  0.47170903262576364
RMSE der Mittelwertsbaseline:  0.4800466959093311


In [17]:
#featureselection
#features to encode
obj_bool_features = accidents[['Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
                         'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
                         'Sunrise_Sunset']]

#encode features above
le = LabelEncoder()
#create copy of df
accidents_encoded = pd.DataFrame()
#LabelEncoder can only encode one column at a time --> forloop
#obj_bool_feat = le.fit_transform(obj_bool_features)
for X in obj_bool_features:
    accidents_encoded[features] = le.fit_transform(accidents[features])

#float/int features
float_int_features = accidents[['Severity','Start_Lng', 'Start_Lat', 'Temperature(F)', 'Humidity(%)', 
                                'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',]]

for X in float_int_features:
    accidents_encoded[features] = accidents[features]

#check if combination worked 
accidents_encoded.columns

ValueError: Boolean array expected for the condition, not float64

In [14]:
# Create a column transformer to handle numeric features
preprocessor = ColumnTransformer(
    transformers=[
        ('densify_vectors', FunctionTransformer(lambda x: x, accept_sparse=True),features),
    ],
    remainder='passthrough'
)

In [16]:
# Pipeline erstellen
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                ('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'), threshold='mean')), 
                ('classifier', GaussianNB())])

# Trainieren der Pipeline
pipeline.fit(features_train, target_train)

# Vorhersagen machen
predicted_dev = pipeline.predict(features_dev)

print("-----------------------------------------------------------------------")
# Bericht für die Pipeline ausgeben
print("Classification Report für die Pipeline (VarianceThreshold + SVC):")
print(metrics.classification_report(y_dev, predicted_dev))


ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed